In [1]:
import numpy as np
import pandas as pd
import itertools
import os

from scipy.stats import wilcoxon

floder_results = "results"

In [2]:
# function to convert settings to file name
def return_file_name(non_linear=False, factors=1, sample=50, α=0, ρ=0.5, n_vars=10, missings=0):
    this_comb = tuple((factors, sample, missings, n_vars, non_linear, ρ, α))
    return str(this_comb).replace(",", "").replace("(", "").replace(")", "").replace(" ", "_")
# function to compute statistics
def compute_stats(df, mo, models, stats):
    assert len(models) == 2
    j_m = models.index(mo)
    if j_m == 0:
        _j_m = 1
    else:
        _j_m = 0
    d = df[mo].values - df[models[_j_m]].values
    series = df[mo].values
    if stats == "median":
        _out = np.median(series)
    elif stats == "mean":
        _out = np.mean(series)
    elif stats == "std":
        _out = np.std(series)
    elif stats == "wilcoxon_stat_two_sided":
        _out = wilcoxon(d)
    elif stats == "wilcoxon_stat_one_sided":
        if np.median(d) > 0:
            _out = wilcoxon(d, alternative="greater")
        else:
            _out = wilcoxon(d, alternative="less")
    return _out

In [3]:
# compute stats in all files
list_done_files = os.listdir("./" + floder_results)
dict_results = dict()
for this_file in list_done_files:
    if this_file[-4:] == ".csv":
        # init dict info
        sum_stat_res_from_csv = {'median': list(), 
                                 'mean': list(), 
                                 'std': list(), 
                                 'wilcoxon_stat_two_sided': list(), 
                                 'wilcoxon_stat_one_sided': list()}
        by_model_sum_stat_res_from_csv = {'dfm': sum_stat_res_from_csv.copy(), 
                                         'ddfm': sum_stat_res_from_csv.copy()}

        # read file
        this_df = pd.read_csv(floder_results + "/" + this_file)
        this_df["dfm"] = np.maximum(this_df["dfm smoothed"].values, this_df["dfm filtered"].values)
        this_df["ddfm"] = np.maximum(this_df["ddfm non-filtered"].values, this_df["ddfm filtered"].values)
        list_models = list(by_model_sum_stat_res_from_csv.keys())
        list_stats = sum_stat_res_from_csv.keys()
        for mo in list_models:
            for stats in list_stats:
                _outt = compute_stats(this_df, mo, list_models, stats)
                by_model_sum_stat_res_from_csv[mo][stats] = _outt
        dict_results[this_file[:-4]] = by_model_sum_stat_res_from_csv

In [5]:
# fill excel files
worksheet = "linear"
non_linear = worksheet=="nonlinear"
n_digits_round = 3
stat = "median"
import openpyxl
xfile = openpyxl.load_workbook('results_summary.xlsx')
sheet = xfile.get_sheet_by_name(worksheet)
def map_diff_to_value_with_sign(to_write_in_diff, wilcoxon_stat):
    if abs(to_write_in_diff) > 0:
        if (wilcoxon_stat <= 0.1)*(wilcoxon_stat>0.05):
            to_write_in_diff = str(to_write_in_diff) + "*"
        elif (wilcoxon_stat <= 0.05)*(wilcoxon_stat > 0.01):
            to_write_in_diff = str(to_write_in_diff) + "**"
        elif wilcoxon_stat <= 0.01:
            to_write_in_diff = str(to_write_in_diff) + "***"
    return to_write_in_diff
for tables in [0, 1]:
    # first 2 sub-tables
    factors = sheet['B'+str(1+(19*tables))].value
    for i in range(4, 20):
        # first sub-table
        sample = sheet['B'+str(2+(19*tables))].value
        alpha = sheet['A'+str(i+(19*tables))].value
        rho = sheet['B'+str(i+(19*tables))].value
        n_vars = sheet['C'+str(i+(19*tables))].value
        missings = sheet['D'+str(i+(19*tables))].value
        this_res = dict_results[return_file_name(non_linear=non_linear, factors=factors, sample=sample, 
                                                 α=alpha, ρ=rho, n_vars=n_vars, missings=missings)]
        sheet['E'+str(i+(19*tables))] = np.round(this_res["ddfm"][stat], n_digits_round)
        sheet['F'+str(i+(19*tables))] = np.round(this_res["dfm"][stat], n_digits_round)
        wilcoxon_stat = this_res['dfm']['wilcoxon_stat_two_sided'][1]
        to_write_in_diff = np.round(this_res["ddfm"][stat], n_digits_round) - np.round(this_res["dfm"][stat], n_digits_round)
        to_write_in_diff = np.round(to_write_in_diff, n_digits_round)
        to_write_in_diff = map_diff_to_value_with_sign(to_write_in_diff, wilcoxon_stat)
        sheet['G'+str(i+(19*tables))] = to_write_in_diff
        # second sub-table
        sample = sheet['H'+str(2+(19*tables))].value
        alpha = sheet['H'+str(i+(19*tables))].value
        rho = sheet['I'+str(i+(19*tables))].value
        n_vars = sheet['J'+str(i+(19*tables))].value
        missings = sheet['K'+str(i+(19*tables))].value
        this_res = dict_results[return_file_name(non_linear=non_linear, factors=factors, sample=sample, 
                                                 α=alpha, ρ=rho, n_vars=n_vars, missings=missings)]
        sheet['L'+str(i+(19*tables))] = np.round(this_res["ddfm"][stat], n_digits_round)
        sheet['M'+str(i+(19*tables))] = np.round(this_res["dfm"][stat], n_digits_round)
        wilcoxon_stat = this_res['dfm']['wilcoxon_stat_two_sided'][1]
        to_write_in_diff = np.round(this_res["ddfm"][stat], n_digits_round) - np.round(this_res["dfm"][stat], n_digits_round)
        to_write_in_diff = np.round(to_write_in_diff, n_digits_round)
        to_write_in_diff = map_diff_to_value_with_sign(to_write_in_diff, wilcoxon_stat)
        sheet['N'+str(i+(19*tables))] = to_write_in_diff
# save file
xfile.save('results_summary.xlsx')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  
